In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [39]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K

import matplotlib.pyplot as plt
import time
import sys

import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

img_rows = 36
img_cols = 36
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

n_critic = 10
clip_value = 0.01
optimizer = RMSprop(lr=0.00001)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
def build_generator():
  model = Sequential()

  # model.add(Dense(128, input_dim=latent_dim))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(BatchNormalization(momentum=0.8))

  # model.add(Dense(256))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(BatchNormalization(momentum=0.8))

  # model.add(Dense(512))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(BatchNormalization(momentum=0.8))

  # model.add(Dense(1024))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(BatchNormalization(momentum=0.8))

  # model.add(Dense(np.prod(img_shape), activation='tanh'))
  # model.add(Reshape(img_shape))

  model.add(Dense(256 * 9 * 9, activation="relu", input_dim=latent_dim))
  model.add(Reshape((9, 9, 256)))
  model.add(UpSampling2D())
  
  model.add(Conv2D(512, kernel_size=4, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  
  model.add(Conv2D(512, kernel_size=4, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  
  model.add(Conv2D(1024, kernel_size=4, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  
  model.add(Conv2D(channels, kernel_size=4, padding="same"))
  model.add(Activation("tanh"))

  model.summary()
  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return model

In [27]:
def build_critic():
  model = Sequential()
  
  # model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same', input_shape=img_shape))
        
  # model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same'))
  # model.add(Dropout(0.25))

  # model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=img_shape))
        
  # model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
  # model.add(Dropout(0.25))
  # model.add(Flatten())

  # model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.5))
  
  # model.add(Dense(1, activation='sigmoid'))

  # ------------------------------------------------------------------------------------------------------------# 

  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))

  model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))

  model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())

  model.add(Dense(1))

  # ------------------------------------------------------------------------------------------------------------# 
  
  # model.add(Flatten(input_shape=img_shape))
  # model.add(Dense(512))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(Dense(256))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(Dense(1, activation='sigmoid'))

  model.summary()
  img = Input(shape=img_shape)
  validity = model(img)
  
  return model

In [40]:
# Wasserstein loss calculation
def wasserstein_loss(y_true, y_pred):
  return K.mean(y_true * y_pred)

# Build and compile the critic
critic = build_critic()
critic.compile(loss=wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generated imgs
z = Input(shape=(latent_dim,))
img = generator(z)
  
# For the combined model we will only train the generator
critic.trainable = False

# The critic takes generated images as input and determines validity
valid = critic(img)

# The combined model  (stacked generator and critic)
combined = Model(z, valid)
combined.compile(loss=wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 18, 18, 32)        320       
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 18, 18, 32)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 9, 9, 64)          18496     
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 10, 10, 64)        0         
_________________________________________________________________
batch_normalization_40 (Batc (None, 10, 10, 64)        256       
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 10, 10, 64)      

In [41]:
def train(epochs, batch_size=128, sample_interval=50):
  read=pd.read_csv("/content/drive/My Drive/CSV_28/33ksh.csv")
  read1 = read.iloc[:,:].values
  arr=np.array
  arr = read1.reshape(725,36,36)
  # Load the dataset
  X_train = arr

  # Rescale -1 to 1
  X_train = X_train / 127.5 - 1.
  X_train = np.expand_dims(X_train, axis=3)

  # Adversarial ground truths
  valid = -np.ones((batch_size, 1))
  fake = np.ones((batch_size, 1))

  for epoch in range(epochs):
    for _ in range(n_critic):
      # Select a random batch of images
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      # Sample noise as generator input
      noise = np.random.normal(0, 1, (batch_size, latent_dim))

      # Generate a batch of new images
      gen_imgs = generator.predict(noise)

      # Train the critic
      d_loss_real = critic.train_on_batch(imgs, valid)
      d_loss_fake = critic.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

      # Clip critic weights
      for l in critic.layers:
        weights = l.get_weights()
        weights = [np.clip(w, -clip_value, clip_value) for w in weights]
        l.set_weights(weights)

    g_loss = combined.train_on_batch(noise, valid)
      # Plot the progress
      #print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))
      # If at save interval => save generated image samples
    accuracy = 0
    if epoch % sample_interval == 0:
      print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))
      accuracy += 100*(1 - d_loss[1])
      sample_images(epoch)
  print("Accuracy: " , accuracy/25)

In [25]:
def sample_images(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
      axs[i,j].axis('off')
      cnt += 1
  fig.savefig("/content/drive/My Drive/WGAN/33ksh_g/33ksh_%d.png" % epoch)
  print("Figure Saved...........!!")
  plt.close()

In [ ]:
if __name__ == '__main__':
    start = time.time()
    train(epochs=5000, batch_size=32, sample_interval=200)
    end = time.time()
    elapsed_train_time = 'elapsed training time: {} min, {} sec '.format(int((end - start) / 60),int((end - start) % 60))
    print(elapsed_train_time)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.999973] [G loss: 1.000075]
200 [D loss: 0.999995] [G loss: 1.000054]
Figure Saved...........!!
400 [D loss: 1.000002] [G loss: 1.000086]
Figure Saved...........!!
600 [D loss: 1.000027] [G loss: 1.000116]
Figure Saved...........!!
